In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from google.colab import drive
drive.mount ('/content/drive')
tf.__version__

Mounted at /content/drive


'2.12.0'

In [2]:
dataset = pd.read_csv('/content/drive/MyDrive/483/Churn_Modelling.csv')
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values


In [4]:
#Encoding
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform (X[:,2])

In [5]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer (transformers= [('encoder', OneHotEncoder(), [1])], remainder = 'passthrough')

In [6]:
X = np.array (ct.fit_transform(X))

In [7]:
#Splitting the dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [8]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [9]:
ann = tf.keras.models.Sequential()

In [10]:
ann.add (tf.keras.layers.Dense (units = 6, activation = 'relu'))

In [11]:
#2nd Hidden Layer addition
ann.add (tf.keras.layers.Dense (units = 6, activation = 'relu'))


In [12]:
ann.add (tf.keras.layers.Dense (units = 1, activation = 'sigmoid'))

In [13]:
ann.compile (optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])


In [14]:
ann.fit (X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 2s 2ms/step - loss: 0.6941 - accuracy: 0.5770
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.5031 - accuracy: 0.8008
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4560 - accuracy: 0.8108
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4357 - accuracy: 0.8161
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4275 - accuracy: 0.8189
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4220 - accuracy: 0.8207
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4176 - accuracy: 0.8210
Epoch 8/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4128 - accuracy: 0.8206
Epoch 9/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4095 - accuracy: 0.8226
Epoch 10/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4055 - accura

In [15]:
#prediction
print (ann.predict(sc.transform ([[1,0,0,600,1,40,3,60000,2,1,1,5000]])) > 0.5)

1/1 [==============================] - 0s 112ms/step
[[False]]


In [16]:
#Predict France, Score = 600, Male, 40 Y old, 3 years tenure, Bal = 60000, Pr_num = 2, credit card = yes, active member = yes, Est Salary = 50000, say good bye or not
print (ann.predict(sc.transform ([[1,0,0,600,1,40,3,60000,2,1,1,5000]])) > 0.5)

1/1 [==============================] - 0s 124ms/step
[[False]]


In [18]:
y_pred = ann.predict (X_test)
y_pred = (y_pred > 0.5)
print (np.concatenate ((y_pred.reshape (len(y_pred),1), y_test.reshape (len(y_test), 1)),1))

63/63 [==============================] - 0s 1ms/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


In [21]:
#Confusion Matrix
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix (y_test, y_pred)
print (cm)
accuracy_score (y_test, y_pred)

[[1509   86]
 [ 200  205]]


0.857